In [163]:
import xml.etree.ElementTree as  ET
import pandas as pd

query = []
accession = []
identity = []
sciname = []


path = "blast_output.xml"
for event, item in ET.iterparse(path,events=("start", "end")):
    if item.tag == 'Iteration_query-def' and event == "start":
        query.append(item.text)
    if item.tag == 'Hit' and event == "start":
        if item[0].text == '1':
            accession.append(item[3].text) #accession
            identity.append(int(item[5][0][10].text)/int(item[5][0][13].text))
            sciname.append(item[2].text)

df = pd.DataFrame([query, accession, identity, sciname]).transpose()
df.columns = ['query', 'accession', 'identity', 'sciname']
df.to_csv("output.csv",index = False)

In [3]:
from Bio.Blast import NCBIWWW

path = "data\\fasta_file\\output\\LWW016_all_1.fas"
outpath = "data\\blast_result\\blast_output.xml"

def Blast_fasta(path,outpath):

    with open (path) as obj:
        fasta_file = obj.read()

    blast_result = NCBIWWW.qblast("blastn", "nt", fasta_file, megablast=True, hitlist_size=10)
    out = blast_result.read()

    with open(outpath, "w") as output_xml:
        output_xml.write(out)

    blast_result.close()

In [1]:
import os

filelist = os.listdir("data\\fasta_file\\output")

for item in filelist:
    path = "data\\fasta_file\\output\\" + item
    outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
    try:
        Blast_fasta(path,outpath)
    except:
        pass

In [1]:
import re
outfilelist = []
outfile = os.listdir("data\\blast_result\\output")
pattern = r"[_][0-9]+"
for item in outfile:
    result = re.search(pattern,item)
    if result:
        outfilelist.append(item[result.start():result.end()])

templist = []
import os
filelist = os.listdir("data\\fasta_file\\output")
for item in filelist:
    result = (re.search(pattern,item))
    temp = item[result.start():result.end()]
    if temp in outfilelist:
        pass
    else:
        path = "data\\fasta_file\\output\\" + item
        outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
        try:
            Blast_fasta(path,outpath)
            print(item + " Blastn success!")
            print("-"*10)
        except:
            pass

In [ ]:
result_dict = {}
temp_query = ""
accession = []
identity = []
sciname = []
count = 0


import xml.etree.ElementTree as  ET
import pandas as pd
import os

filelist = os.listdir("data\\blast_result\\output")
filelist = sorted(filelist, reverse=True)

for i in filelist:
    path = "data\\blast_result\\output\\" + i
    for event, item in ET.iterparse(path,events=("start", "end")):
        if item.tag == 'Iteration_query-def' and event == "start":
            temp_query = item.text
        if item.tag == 'Hit' and event == "start":
            try:
                if item[0].text == '1':
                    count+=1
                    try:
                        result_dict[temp_query] = [item[3].text] #accession
                    except:
                        pass
                    try:
                        result_dict[temp_query].append(f"{(int(item[5][0][10].text)/int(item[5][0][13].text)):.2%}") #identity
                    except:
                        pass
                    try:
                        result_dict[temp_query].append(item[2].text) #sci_name
                    except:
                        pass
            except:
                print("query " + str(count) + " error")
                count+=1
                pass